In [1]:
import torchvision.models as models
from src import util


resnet = models.resnet18(pretrained=True)

In [2]:
print(resnet.layer4[1].conv2.weight)
# print(resnet.layer4[1].bn1.weight)
util.rand_layers(resnet, [['fc'], ['layer4', '1']])
print(resnet.layer4[1].conv2.weight)
# print(resnet.layer4[1].bn1.weight)

Parameter containing:
tensor([[[[ 2.8729e-04,  4.2632e-03, -2.0266e-03],
          [ 1.9513e-04,  2.4381e-03, -5.8632e-03],
          [ 4.4803e-03,  8.6577e-03,  8.5538e-04]],

         [[-1.1335e-02, -1.3195e-02, -1.0305e-02],
          [-4.9507e-03, -4.5898e-03, -3.1041e-03],
          [-7.5883e-03, -8.3795e-03, -8.9239e-03]],

         [[-1.1914e-02, -1.2104e-02, -1.0167e-02],
          [-1.2093e-02, -1.1557e-02, -8.9600e-03],
          [-1.2515e-02, -9.3296e-03, -6.4079e-03]],

         ...,

         [[-9.3573e-03, -1.0662e-02, -1.2672e-02],
          [-8.0600e-03, -8.5423e-03, -1.2121e-02],
          [-8.1498e-03, -8.8037e-03, -1.0611e-02]],

         [[ 4.2632e-03,  5.6461e-03,  2.8460e-03],
          [ 4.7070e-03,  6.2550e-03,  7.5862e-03],
          [ 1.1504e-02,  1.1518e-02,  1.0728e-02]],

         [[-6.2455e-03, -9.1693e-03, -9.6664e-03],
          [-4.2935e-03, -6.5311e-03, -5.0513e-03],
          [-3.1141e-03, -5.0124e-03, -5.8122e-03]]],


        [[[ 2.7483e-03,  3.7146

NameError: name 'util' is not defined

In [3]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for k in resnet.modules():
    if isinstance(k, torch.nn.modules.container.Sequential):
        print(k.children())
        print(k.downsample)
#         for l in k.modules():
#             print(l)
        
print(type(resnet.layer4[0].conv1))